## IMPORT

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 462 kB 43.8 MB/s 
     |████████████████████████████████| 4.2 MB 21.8 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import pandas as pd

import tensorflow_recommenders as tfrs

# getting data
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


In [4]:
print(os.getcwd())

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0


## LOAD DATA FROM MYSQL

In [16]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

# db_connection = 'mysql+pymysql://root:@localhost/notogo_database'
# conn = create_engine(db_connection)

# df = pd.read_sql("select * from tab_name", conn)

db_connect =pymysql.connect(host='127.0.0.1',user='root',password='',db='notogo_database')

# df=pd.read_sql_query("SELECT * FROM 'YOUR_TABLENAME' ",conn)

OperationalError: ignored

In [17]:
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server-5.7 mysql-server-core-5.7
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl li

In [ ]:
db_connect = sql.connect(host = '127.0.0.1', database = 'notogo_database', user = 'root', password = '')

In [ ]:
dataset = tf.data.experimental.SqlDataset("mysql", "/foo/bar.sqlite3",
                                          "SELECT name, age FROM people",
                                          (tf.string, tf.int32))

## Preparing the dataset

In [5]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incompleteSP77CI/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [6]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incomplete88MPF4/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [7]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x["like"]
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

In [8]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0, 'like': 1, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'like': 1, 'location_name': b'DUBAI', 'user_id': b'112'}


In [9]:
tf.random.set_seed(42)

NUM_DATA = ratings.__len__().numpy()

shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [ ]:
# class NoToGoModel(tfrs.models.Model):

#   def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
#     # We take the loss weights in the constructor: this allows us to instantiate
#     # several model objects with different loss weights.

#     super().__init__()

#     embedding_dimension = 32

#     # User and movie models.
#     self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
#       tf.keras.layers.StringLookup(
#         vocabulary=unique_location_name, mask_token=None),
#       tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
#       tf.keras.layers.Dense(32, activation="relu")
#     ])
#     self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
#       tf.keras.layers.StringLookup(
#         vocabulary=unique_user_ids, mask_token=None),
#       tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
#       tf.keras.layers.Dense(32, activation="relu")
#     ])

#     # A small model to take in user and movie embeddings and predict ratings.
#     # We can make this as complicated as we want as long as we output a scalar
#     # as our prediction.
#     self.rating_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(16, activation="relu"),
#         tf.keras.layers.Dense(64, activation="relu"),
#         tf.keras.layers.Dense(1, activation = "sigmoid"),
#     ])

#     # The tasks.
#     self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
#         loss=tf.keras.losses.MeanSquaredError(),
#         metrics=[tf.keras.metrics.RootMeanSquaredError()],
#     )
#     self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
#         metrics=tfrs.metrics.FactorizedTopK(
#             candidates=locations.batch(128).map(self.location_model)
#         )
#     )

#     # The loss weights.
#     self.rating_weight = rating_weight
#     self.retrieval_weight = retrieval_weight

#   def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     # And pick out the movie features and pass them into the movie model.
#     location_embeddings = self.location_model(features["location_name"])
    
#     return (
#         user_embeddings,
#         location_embeddings,
#         # We apply the multi-layered rating model to a concatentation of
#         # user and movie embeddings.
#         self.rating_model(
#             tf.concat([user_embeddings,location_embeddings], axis=1)
#         ),
#     )

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

#     ratings = features.pop("add")

#     user_embeddings, location_embeddings, rating_predictions = self(features)

#     # We compute the loss for each task.
#     rating_loss = self.rating_task(
#         labels=ratings,
#         predictions=rating_predictions,
#     )
#     retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

#     # And combine them using the loss weights.
#     return (self.rating_weight * rating_loss
#             + self.retrieval_weight * retrieval_loss)

In [10]:
class NoToGoModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, like_weight: float,retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    self.like_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    self.like_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight
    self.like_weight = like_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
        self.like_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add","like")
    like = features.pop("like", "add")

    user_embeddings, location_embeddings, rating_predictions, like_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )

    like_loss = self.like_task(
        labels=like,
        predictions=like_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss + like_loss*self.like_weight)

### SKIP

In [ ]:
testModel = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, 32),
      tf.keras.layers.Dense(32, activation="relu")
])

In [ ]:
cek = tf.keras.layers.StringLookup(vocabulary=unique_location_name, mask_token=None)

In [ ]:
unique_location_name

array([b'AIR TERJUN SIPISO-PISO', b'AKIHABARA', b'ALASKA',
       b'AMERIKA SERIKAT', b'ANCOL', b'ASIA TIMUR', b'BALI',
       b'BALI TOWER', b'BANDUNG', b'BANGKA BELITUNG', b'BHUTAN',
       b'BRANCHSTO BSD', b'CANDI BOROBUDUR', b'CANDI PRAMBANAN',
       b'CANGGU', b'CAPADOCIA', b'DANAU SALAR DE UYUNI', b'DANAU TOBA',
       b'DENMARK', b'DIENG', b'DISNEY SEA', b'DISNEYLAND PARIS', b'DUBAI',
       b'DUFAN', b'EROPA', b'ES KRIM RAGUSA', b'EUROPE',
       b'GAMPLONG STUDIO', b'GEOPARK CILETUH', b'GUA GONG',
       b'GUA TABUHAN', b'GUMUK PASIR PARANGKUSUMO', b'GUNUNG',
       b'GUNUNG AGUNG', b'GUNUNG BROMO', b'GUNUNG FUJI', b'GUNUNG GEDE',
       b'GUNUNG PANGRANGO', b'GUNUNG PARANG', b'GUNUNG RINJANI',
       b'GUNUNG SEMERU', b'INDONESIA', b'ISRAEL', b'JAKARTA',
       b'JAWA BARAT', b'JEPANG', b'JERMAN', b'KATULAMPA RAFTING BOGOR',
       b'KENDARI', b'KOREA SELATAN', b'LABUAN BAJO', b'LAMPUNG',
       b'LOMBOK', b'LONDON', b'MALANG', b'MALAYSIA', b'MALDIVES',
       b'MALIOBORO',

In [ ]:
cek(tf.constant([['GUNUNG BROMO','DANAU TOBA']])) # 0 means out of vocabulary

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[35, 18]])>

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [28]:
model = NoToGoModel(rating_weight=1.0,like_weight = 0,retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [13]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [29]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 5s 80ms/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0315 - factorized_top_k/top_10_categorical_accuracy: 0.0714 - factorized_top_k/top_50_categorical_accuracy: 0.3940 - factorized_top_k/top_100_categorical_accuracy: 0.8136 - loss: 0.2093 - regularization_loss: 0.0000e+00 - total_loss: 0.2093
Epoch 2/3
43/43 [==============================] - 3s 80ms/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0027 - factorized_top_k/top_5_categorical_accuracy: 0.0353 - factorized_top_k/top_10_categorical_accuracy: 0.0758 - factorized_top_k/top_50_categorical_accuracy: 0.3992 - factorized_top_k/top_100_categorical_accuracy: 0.8233 - loss: 0.2001 - regularization_loss: 0.0000e+00 - total_loss: 0.2001
Epoch 3/3
22/22 [==============================] - 2s 73ms/step - root_mean_squared_error: 0.0000e+00 - fact

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [14]:
model = NoToGoModel(rating_weight=0.0, like_weight = 0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [15]:
model.fit(cached_train, epochs=50)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/50
43/43 [==============================] - 8s 93ms/step - root_mean_squared_error: 0.5029 - factorized_top_k/top_1_categorical_accuracy: 0.1848 - factorized_top_k/top_5_categorical_accuracy: 0.4426 - factorized_top_k/top_10_categorical_accuracy: 0.4906 - factorized_top_k/top_50_categorical_accuracy: 0.6988 - factorized_top_k/top_100_categorical_accuracy: 0.8854 - loss: 3180.3646 - regularization_loss: 0.0000e+00 - total_loss: 3180.3646
Epoch 2/50
43/43 [==============================] - 7s 168ms/step - root_mean_squared_error: 0.5015 - factorized_top_k/top_1_categorical_accuracy: 0.3514 - factorized_top_k/top_5_categorical_accuracy: 0.7608 - factorized_top_k/top_10_categorical_accuracy: 0.8535 - factorized_top_k/top_50_categorical_accuracy: 0.9992 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3180.2920 - regularization_loss: 0.0000e+00 - total_loss: 3180.2920
Epoch 3/50
43/43 [==============================] - 4s 83ms/step - root_mean_squared_error: 0.5022 - 

In [16]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 2s 73ms/step - root_mean_squared_error: 0.5062 - factorized_top_k/top_1_categorical_accuracy: 0.3494 - factorized_top_k/top_5_categorical_accuracy: 0.4697 - factorized_top_k/top_10_categorical_accuracy: 0.4841 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1347.5984 - regularization_loss: 0.0000e+00 - total_loss: 1347.5984


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.48407062888145447,
 'factorized_top_k/top_1_categorical_accuracy': 0.34935373067855835,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.46968868374824524,
 'loss': 559.3197631835938,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5062288641929626,
 'total_loss': 559.3197631835938}

In [17]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)
# Get recommendations.
user_id = "183"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for Dimas : {titles[0, :7]}")

Recommendations for Dimas : [b'ASIA TIMUR' b'JAKARTA' b'BHUTAN' b'BANGKA BELITUNG' b'MALIOBORO'
 b'MALIOBORO' b'GUNUNG BROMO']


### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 4s 69ms/step - root_mean_squared_error: 0.4551 - factorized_top_k/top_1_categorical_accuracy: 0.0099 - factorized_top_k/top_5_categorical_accuracy: 0.0650 - factorized_top_k/top_10_categorical_accuracy: 0.1283 - factorized_top_k/top_50_categorical_accuracy: 0.5032 - factorized_top_k/top_100_categorical_accuracy: 0.8166 - loss: 3176.1545 - regularization_loss: 0.0000e+00 - total_loss: 3176.1545
Epoch 2/3
43/43 [==============================] - 3s 71ms/step - root_mean_squared_error: 0.4460 - factorized_top_k/top_1_categorical_accuracy: 0.0100 - factorized_top_k/top_5_categorical_accuracy: 0.0776 - factorized_top_k/top_10_categorical_accuracy: 0.2163 - factorized_top_k/top_50_categorical_accuracy: 0.6746 - factorized_top_k/top_100_categorical_accuracy: 0.9475 - loss: 3176.1255 - regularization_loss: 0.0000e+00 - total_loss: 3176.1255
Epoch 3/3
22/22 [==============================] - 2s 61ms/step - root_mean_squared_error: 0.4467 - fact

## INTERFACE FOR NEW USER

In [ ]:
worksheetCat = gc.open('capstone_dataset').worksheet('category')
rows = worksheetCat.get_all_values()
dfCategories = pd.DataFrame.from_records(rows[1:], columns=rows[0])
list_categories = [i for i in dfCategories['category'] if i != '']
worksheetUserFeat = gc.open('capstone_dataset').worksheet('userFeatures(coldstartsol)')
rows = worksheetUserFeat.get_all_values()
dfUserFeat = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
last_user_id = dfUserFeat['user_id'].to_list()
last_user_id = int(last_user_id[-1]) + 1 

def input_user_data():
  user_data = {}
  # input_name = input("Nama: ")
  user_data['user_id'] = str(last_user_id)
  last_user_ids = last_user_id + 1
  user_categories = list()
  for i in list_categories:
    input_category = int(input("{} ? (1: yes, 0: no) : ".format(i)))
    if input_category == 1:
      user_categories.append(i)
  user_data["categories"] = user_categories
  return user_data, last_user_ids
user_data, last_user_id = input_user_data()

In [ ]:
def encode_dummy_category(location):
  if location == "beach":
    return 121
  if location == "mountain":
    return 123
  if location == "lake":
    return 124
  if location == "zoo":
    return 125
  if location == "river":
    return 126
  if location == "conservation":
    return 127
  if location == "waterpark":
    return 128
  if location == "waterfall":
    return 129
  if location == "artGallery":
    return 130
  if location == "amusementPark":
    return 131
  if location == "mall":
    return 132
  if location == "HistoricalPlace":
    return 133
  if location == "religious":
    return 134
  if location == "outbond":
    return 135
  if location == "culinary":
    return 136
  if location == "photoHunting":
    return 137
  if location == "sightSeeing":
    return 138
  if location == "shopping":
    return 139

In [ ]:
def create_data_frame(user_data, dfUserFeatcold):
  user_id = [user_data['user_id'] for i in list_categories]
  like = ['0' for i in list_categories]
  categories = list_categories
  location = categories
  location_id = [encode_dummy_category(i) for i in list_categories]
  add = list()
  for i in list_categories:
    if i in user_data['categories']:
      add.append("1")
    else:
      add.append("0")
  data = {'user_id' : user_id, 'like' : like, "add" : add, "category" : categories, "location" : location, "location_id" : location_id}
  df = pd.DataFrame(data)
  df_concat = pd.concat([dfUserFeatcold,df], ignore_index= True)
  return df, df_concat   


In [ ]:
df, df_concat = create_data_frame(user_data, dfUserFeat)
worksheetUser = gc.open('capstone_dataset').worksheet('userFeatures')
set_with_dataframe(worksheetUser, df_concat)